In [18]:
import numpy as np
import matplotlib
from matplotlib import colors
import matplotlib.pyplot as plt
import geone
import geone.covModel as gcm
import geone.imgplot3d as imgplt3
import pyvista as pv
import sys
import os
import flopy as fp
import pyemu

sys.path.append("../../../")
#my modules
from ArchPy.base import *
from ArchPy.tpgs import *

In [19]:
tmp_model_ws = "reference"
sim = fp.mf6.MFSimulation.load(sim_ws=tmp_model_ws)
m = sim.get_model("test")

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package oc...
    loading package ic...
    loading package npf...
    loading package chd...
    loading package obs...
  loading solution package test...


In [20]:
sr = pyemu.helpers.SpatialReference.from_namfile(
        os.path.join(tmp_model_ws, "test.nam"),
        delr=m.dis.delr.array, delc=m.dis.delc.array)
sr
template_ws = "template"
pf = pyemu.utils.PstFrom(original_d=tmp_model_ws, new_d=template_ws,
                 remove_existing=True,
                 longnames=True, spatial_reference=sr,
                 zero_based=False,start_datetime="1-1-2025")

   could not remove start_datetime
2025-09-04 12:29:45.128287 starting: opening PstFrom.log for logging
2025-09-04 12:29:45.128287 starting PstFrom process
2025-09-04 12:29:45.129250 starting: setting up dirs
2025-09-04 12:29:45.130250 starting: removing existing new_d 'template'
2025-09-04 12:29:45.131248 finished: removing existing new_d 'template' took: 0:00:00.000998
2025-09-04 12:29:45.131248 starting: copying original_d 'reference' to new_d 'template'
2025-09-04 12:29:45.166591 finished: copying original_d 'reference' to new_d 'template' took: 0:00:00.035343
2025-09-04 12:29:45.167597 finished: setting up dirs took: 0:00:00.038347


In [21]:
tmp_model_ws

'reference'

## Observations

In [22]:
df = pd.read_csv(os.path.join(tmp_model_ws,"head_obs_ref.csv"),index_col=0)
df

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
time,,,,,,,,,,
1.0,0.73881,0.425,0.42667,0.11038,0.025822,0.037492,0.17995,0.048933,0.79004,0.26967


In [23]:
# open(os.path.join(tmp_model_ws,"head_obs_ref.csv"))

In [24]:
hds_df = pf.add_observations("head_obs_ref.csv",insfile="head_obs_ref.csv.ins", index_cols="time",
                    use_cols=list(df.columns.values),prefix="hds",)
hds_df

2025-09-04 12:29:45.303524 starting: adding observations from output file head_obs_ref.csv
2025-09-04 12:29:45.304548 starting: adding observations from tabular output file '['head_obs_ref.csv']'
2025-09-04 12:29:45.304548 starting: reading list-style file: template\head_obs_ref.csv
2025-09-04 12:29:45.310541 finished: reading list-style file: template\head_obs_ref.csv took: 0:00:00.005993
2025-09-04 12:29:45.314538 starting: building insfile for tabular output file head_obs_ref.csv
2025-09-04 12:29:45.325542 finished: building insfile for tabular output file head_obs_ref.csv took: 0:00:00.011004
2025-09-04 12:29:45.325542 starting: adding observation from instruction file 'template\head_obs_ref.csv.ins'
2025-09-04 12:29:45.335623 finished: adding observation from instruction file 'template\head_obs_ref.csv.ins' took: 0:00:00.010081
2025-09-04 12:29:45.336572 finished: adding observations from tabular output file '['head_obs_ref.csv']' took: 0:00:00.032024
2025-09-04 12:29:45.336572 fi

,obsnme,obsval,weight,obgnme
oname:hds_otype:lst_usecol:h10_time:1,oname:hds_otype:lst_usecol:h10_time:1,0.269670,1.0,oname:hds_otype:lst_usecol:h10
oname:hds_otype:lst_usecol:h1_time:1,oname:hds_otype:lst_usecol:h1_time:1,0.738810,1.0,oname:hds_otype:lst_usecol:h1
oname:hds_otype:lst_usecol:h2_time:1,oname:hds_otype:lst_usecol:h2_time:1,0.425000,1.0,oname:hds_otype:lst_usecol:h2
oname:hds_otype:lst_usecol:h3_time:1,oname:hds_otype:lst_usecol:h3_time:1,0.426670,1.0,oname:hds_otype:lst_usecol:h3
oname:hds_otype:lst_usecol:h4_time:1,oname:hds_otype:lst_usecol:h4_time:1,0.110380,1.0,oname:hds_otype:lst_usecol:h4
oname:hds_otype:lst_usecol:h5_time:1,oname:hds_otype:lst_usecol:h5_time:1,0.025822,1.0,oname:hds_otype:lst_usecol:h5
oname:hds_otype:lst_usecol:h6_time:1,oname:hds_otype:lst_usecol:h6_time:1,0.037492,1.0,oname:hds_otype:lst_usecol:h6
oname:hds_otype:lst_usecol:h7_time:1,oname:hds_otype:lst_usecol:h7_time:1,0.179950,1.0,oname:hds_otype:lst_usecol:h7
oname:hds_otype:lst_usecol:h8_time:1,oname:hds_otype:lst_usecol:h8_time:1,0.048933,1.0,oname:hds_otype:lst_usecol:h8
oname:hds_otype:lst_usecol:h9_time:1,oname:hds_otype:lst_usecol:h9_time:1,0.790040,1.0,oname:hds_otype:lst_usecol:h9


In [25]:
[f for f in os.listdir(template_ws) if f.endswith(".ins")]

['head_obs_ref.csv.ins']

## Parameters

parameters are the surface elevation of unit C, stored as a txt file

In [26]:
pf

In [27]:
arr_file = "Surf_C.txt"
tag = arr_file.split('.')[0]
pf.add_parameters(filenames=arr_file, par_type="grid",
                   par_name_base=tag,pargp=tag,zone_array=None,
                   par_style="multiplier",
                   upper_bound=10.,lower_bound=0.1,ult_ubound=100,ult_lbound=0.01)

2025-09-04 12:29:45.390550 transform was not passed, setting default tranform to 'log'
2025-09-04 12:29:45.390550 starting: adding grid type m style parameters for file(s) ['Surf_C.txt']
2025-09-04 12:29:45.391549 starting: loading array template\Surf_C.txt
2025-09-04 12:29:45.422067 finished: loading array template\Surf_C.txt took: 0:00:00.030518
2025-09-04 12:29:45.422067 loaded array 'reference\Surf_C.txt' of shape (70, 140)
2025-09-04 12:29:45.433067 starting: writing array-style template file 'template\surf_c_inst0_grid.csv.tpl'
2025-09-04 12:29:45.433067 starting: writing template file template\surf_c_inst0_grid.csv.tpl for ['surf_c_inst:0']
2025-09-04 12:29:45.567890 finished: writing template file template\surf_c_inst0_grid.csv.tpl for ['surf_c_inst:0'] took: 0:00:00.134823
2025-09-04 12:29:45.567890 starting: writing array-based template file 'template\surf_c_inst0_grid.csv.tpl'
2025-09-04 12:29:45.586346 finished: adding grid type m style parameters for file(s) ['Surf_C.txt']

,parnme,parval1,i,j,x,y,pargp,tpl_filename,input_filename,partype,partrans,parubnd,parlbnd,scale,offset,parchglim,dercom
pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:0_x:0.75_y:104.25,pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:0_x:0.75_y:104.25,1.0,0,0,0.75,104.25,surf_c,template\surf_c_inst0_grid.csv.tpl,template\mult\surf_c_inst0_grid.csv,grid,log,10.0,0.1,1.0,0.0,factor,1
pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:1_x:2.25_y:104.25,pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:1_x:2.25_y:104.25,1.0,0,1,2.25,104.25,surf_c,template\surf_c_inst0_grid.csv.tpl,template\mult\surf_c_inst0_grid.csv,grid,log,10.0,0.1,1.0,0.0,factor,1
pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:2_x:3.75_y:104.25,pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:2_x:3.75_y:104.25,1.0,0,2,3.75,104.25,surf_c,template\surf_c_inst0_grid.csv.tpl,template\mult\surf_c_inst0_grid.csv,grid,log,10.0,0.1,1.0,0.0,factor,1
pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:3_x:5.25_y:104.25,pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:3_x:5.25_y:104.25,1.0,0,3,5.25,104.25,surf_c,template\surf_c_inst0_grid.csv.tpl,template\mult\surf_c_inst0_grid.csv,grid,log,10.0,0.1,1.0,0.0,factor,1
pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:4_x:6.75_y:104.25,pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:4_x:6.75_y:104.25,1.0,0,4,6.75,104.25,surf_c,template\surf_c_inst0_grid.csv.tpl,template\mult\surf_c_inst0_grid.csv,grid,log,10.0,0.1,1.0,0.0,factor,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pname:surf_c_inst:0_ptype:gr_pstyle:m_i:69_j:135_x:203.25_y:0.75,pname:surf_c_inst:0_ptype:gr_pstyle:m_i:69_j:135_x:203.25_y:0.75,1.0,69,135,203.25,0.75,surf_c,template\surf_c_inst0_grid.csv.tpl,template\mult\surf_c_inst0_grid.csv,grid,log,10.0,0.1,1.0,0.0,factor,1
pname:surf_c_inst:0_ptype:gr_pstyle:m_i:69_j:136_x:204.75_y:0.75,pname:surf_c_inst:0_ptype:gr_pstyle:m_i:69_j:136_x:204.75_y:0.75,1.0,69,136,204.75,0.75,surf_c,template\surf_c_inst0_grid.csv.tpl,template\mult\surf_c_inst0_grid.csv,grid,log,10.0,0.1,1.0,0.0,factor,1
pname:surf_c_inst:0_ptype:gr_pstyle:m_i:69_j:137_x:206.25_y:0.75,pname:surf_c_inst:0_ptype:gr_pstyle:m_i:69_j:137_x:206.25_y:0.75,1.0,69,137,206.25,0.75,surf_c,template\surf_c_inst0_grid.csv.tpl,template\mult\surf_c_inst0_grid.csv,grid,log,10.0,0.1,1.0,0.0,factor,1
pname:surf_c_inst:0_ptype:gr_pstyle:m_i:69_j:138_x:207.75_y:0.75,pname:surf_c_inst:0_ptype:gr_pstyle:m_i:69_j:138_x:207.75_y:0.75,1.0,69,138,207.75,0.75,surf_c,template\surf_c_inst0_grid.csv.tpl,template\mult\surf_c_inst0_grid.csv,grid,log,10.0,0.1,1.0,0.0,factor,1


## Command line

It will be necessary to add instruction to update the modflow model using ArchPy. First new surfaces are loaded into an ArchPy model created based on ArchPy_workspace. Then, archpy2modflow is created, new grid info are retrieved and applied with flopy to modflow model. In addition we have to modify BCs and obs to match new grid...
 inally, modflow 6 can be run.

In [ ]:
# ARCHPY # TO DO

pf.mod_sys_cmds.append("mf6")  # add modflow command
pf.pre_py_cmds.insert(0,"import sys")
# pf.pre_py_cmds.insert(1,"sys.path.append(os.path.join('..','..','..','pypestutils'))")
pf.pre_py_cmds.insert(1,"sys.path.append(os.path.join('..','..', '..', '..'))")
pf.pre_py_cmds.insert(2,"import ArchPy")
pf.mod_py_cmds.append('T1 = ArchPy.inputs.import_project("P1", "../ArchPy_workspace")')
pf.mod_py_cmds.append('surf_C = np.loadtxt("surf_C.txt")')
pf.mod_py_cmds.append('T1.Geol.surfaces_by_piles["P1"][0, 1] = surf_C')
pf.mod_py_cmds.append('archpy_flow = archpy2modflow(T1, exe_name="../../../../../exe/mf6.exe", model_dir="template")')
pf.mod_py_cmds.append('archpy_flow.create_sim(grid_mode="layers", iu=0)')
pst = pf.build_pst()

noptmax:0, npar_adj:9800, nnz_obs:10
2025-09-04 12:29:46.077766 forward_run line:pyemu.os_utils.run(r'mf6')



In [2]:
T1 = ArchPy.inputs.import_project("P1", "ArchPy_workspace")

### IMPORTING PROJECT P1 IN ArchPy_workspace DIRECTORY ### 

Surface_1_dic_surf_mean.npy
Unit A: Surface added for interpolation
Facies basement added to unit A ✅
Unit B: Surface added for interpolation
Facies Sand added to unit B ✅
Facies Gravel added to unit B ✅
Facies GM added to unit B ✅
Facies SM added to unit B ✅
Unit C: Surface added for interpolation
Facies Clay added to unit C ✅
Facies Silt added to unit C ✅
The SubPile object is not an Arch_table object
Unit D: Surface added for interpolation
Facies Sand added to unit D ✅
Facies SM added to unit D ✅
Unit D1: Surface added for interpolation
Facies Sand added to unit D1 ✅
Unit D2: Surface added for interpolation
Facies SM added to unit D2 ✅
Stratigraphic unit D added ✅
Stratigraphic unit C added ✅
Stratigraphic unit B added ✅
Stratigraphic unit A added ✅
Stratigraphic unit D2 added ✅
Stratigraphic unit D1 added ✅
Pile sets as Pile master
Property K added
## Adding Grid ##
## Grid added and is now simulation grid ##


 ##LOADING

In [3]:
T1.verbose=0

In [4]:
T1.compute_surf(1)

In [ ]:
# np.savetxt("template/surf_C.txt", T1.get_surfaces_unit(T1.get_unit("C"))[0])

In [ ]:
# load new surfaces
surf_C = np.loadtxt("template/surf_C.txt")

In [25]:
T1.get_pile_master().list_units

[D, C, B, A]

In [30]:
# add this surface to pile
T1.Geol.surfaces_by_piles["P1"][0, 1] = surf_C

In [7]:
import flopy as fp
import ArchPy.ap_mf
from ArchPy.ap_mf import archpy2modflow, array2cellids

In [8]:
archpy_flow = archpy2modflow(T1, exe_name="../../../../../exe/mf6.exe", model_dir="template")  # create the modflow model
archpy_flow.create_sim(grid_mode="layers", iu=0)  # create the simulation object and choose a certain discretization

Simulation created with the following parameters:
Grid mode: layers
To retrieve the simulation, use the get_sim() method
